In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from model import NERModel, BERTWordEncoder

In [3]:
tensors = [torch.tensor([1,2,3]), torch.tensor([4,5])]
torch.cat(tensors, 0)

tensor([1, 2, 3, 4, 5])

In [1]:
a = [1, 2, 3]
a.clear()
a

[]

In [2]:
lcs = nn.Sequential()
lcs.add_module('lc1', nn.Linear(2,1))
lcs

Sequential(
  (lc1): Linear(in_features=2, out_features=1, bias=True)
)

In [43]:
a, b, c = torch.Tensor([[1,2], [3,4]]), torch.Tensor([[5,6], [7,8]]), torch.Tensor([[9,10], [11,12]])
d = [a, b, c]
pred = torch.Tensor([0,1,1])
logits_0 = torch.stack(d)[:, 0, :]
candidates = logits_0[torch.where(pred == 1)[0]][:, 1]
_, tmp_pred = torch.max(candidates, 0)
torch.where(pred == 1)[0][tmp_pred]

tensor(2)

In [21]:
a = torch.Tensor([0,0,2,2,0,0,2])
a[a > 0] = 1
a

tensor([0., 0., 1., 1., 0., 0., 1.])

In [46]:
l = [torch.Tensor([1]), torch.Tensor([2])]
torch.as_tensor(l)

tensor([1., 2.])

In [20]:
lc1 = nn.Linear(2,2)
print(lc1.weight)
print(lc1.bias)
lc2 = nn.Linear(2,3)
print(lc2.weight)
print(lc2.bias)
with torch.no_grad():
    lc2.weight[:lc1.out_features] = lc1.weight
    lc2.bias[:lc1.out_features] = lc1.bias
print(lc2.weight)
print(lc2.bias)
d3_t = torch.as_tensor([[1,2,3], [3,4,5], [5,6,7]])
d3_t[:, :-1]

Parameter containing:
tensor([[ 0.6101, -0.4459],
        [ 0.6340,  0.5142]], requires_grad=True)
Parameter containing:
tensor([ 0.6353, -0.0373], requires_grad=True)
Parameter containing:
tensor([[-0.3818, -0.5782],
        [ 0.1913,  0.3555],
        [-0.1121, -0.3567]], requires_grad=True)
Parameter containing:
tensor([-0.6889, -0.4358, -0.3898], requires_grad=True)
Parameter containing:
tensor([[ 0.6101, -0.4459],
        [ 0.6340,  0.5142],
        [-0.1121, -0.3567]], requires_grad=True)
Parameter containing:
tensor([ 0.6353, -0.0373, -0.3898], requires_grad=True)


tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [10]:
if None != 'none':
    print(2)

2


In [2]:
word_encoder = BERTWordEncoder('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
class BertTagger(NERModel):
    
    def __init__(self, word_encoder):
        NERModel.__init__(self, word_encoder)
        self.lc_1 = nn.Linear(in_features=word_encoder.output_dim, out_features=10)
        self.drop = nn.Dropout()

    def train_forward(self, x):
        return self.forward(x)

    def forward(self, x):
        embedding = self.encode(x)
        logits = self.lc_1(embedding)  # [num_of_tokens, class_num]
        _, pred = torch.max(logits, 1)
        return logits, pred

In [6]:
model = BertTagger(word_encoder)
print([(n, p) for n, p in model.named_parameters() if n.startswith('lc')])
torch.save({'state_dict': model.state_dict()}, 'lc_1')

[('lc_1.weight', Parameter containing:
tensor([[ 2.8732e-02, -3.5536e-02, -1.8149e-02,  5.2805e-03,  2.0194e-04,
          1.6771e-02, -8.1999e-03, -3.2431e-02,  8.2918e-03,  3.5326e-02,
         -2.0378e-02,  3.4141e-03, -2.3877e-03, -3.0609e-02, -2.5336e-02,
          6.6329e-03, -2.6148e-02, -8.4295e-03,  2.6116e-02,  7.3001e-03,
          1.8179e-03,  8.5523e-03, -8.3591e-03, -2.2094e-02,  1.4133e-02,
          9.4152e-03, -9.1371e-03,  2.6205e-02,  9.9538e-03,  1.9322e-02,
         -2.0262e-02, -2.2829e-02,  1.0747e-02, -4.3306e-04,  6.9762e-03,
         -6.5687e-03,  3.3622e-02, -4.1784e-03, -3.5189e-02,  1.5128e-02,
          1.8027e-02, -1.7781e-02,  3.8927e-04, -2.3789e-02, -1.9742e-02,
          3.5262e-02, -3.4977e-02, -2.3164e-02, -3.4642e-02,  2.9731e-02,
         -3.5738e-03,  8.0636e-04, -1.2149e-03, -4.3577e-03,  3.4910e-02,
          1.6013e-02, -3.9014e-03,  8.7278e-03,  1.1996e-02, -7.4370e-03,
         -1.5086e-02,  3.0152e-02, -6.9149e-04,  3.5315e-02,  1.5711e-02,

In [7]:
class BertTagger_2(NERModel):
    
    def __init__(self, word_encoder):
        NERModel.__init__(self, word_encoder)
        self.lc_1 = nn.Linear(in_features=word_encoder.output_dim, out_features=10)
        self.lc_2 = nn.Linear(in_features=word_encoder.output_dim, out_features=10)
        self.drop = nn.Dropout()

    def train_forward(self, x):
        return self.forward(x)

    def forward(self, x):
        embedding = self.encode(x)
        logits = self.lc_1(embedding)  # [num_of_tokens, class_num]
        _, pred = torch.max(logits, 1)
        return logits, pred

In [8]:
model2 = BertTagger_2(word_encoder)
state_dict = torch.load('lc_1')['state_dict']
own_state = model2.state_dict()
for name, param in state_dict.items():
    if name not in own_state:
        print(name)
        continue
    own_state[name].copy_(param)
print([(n, p) for n, p in model2.named_parameters() if n.startswith('lc')])

[('lc_1.weight', Parameter containing:
tensor([[ 2.8732e-02, -3.5536e-02, -1.8149e-02,  5.2805e-03,  2.0194e-04,
          1.6771e-02, -8.1999e-03, -3.2431e-02,  8.2918e-03,  3.5326e-02,
         -2.0378e-02,  3.4141e-03, -2.3877e-03, -3.0609e-02, -2.5336e-02,
          6.6329e-03, -2.6148e-02, -8.4295e-03,  2.6116e-02,  7.3001e-03,
          1.8179e-03,  8.5523e-03, -8.3591e-03, -2.2094e-02,  1.4133e-02,
          9.4152e-03, -9.1371e-03,  2.6205e-02,  9.9538e-03,  1.9322e-02,
         -2.0262e-02, -2.2829e-02,  1.0747e-02, -4.3306e-04,  6.9762e-03,
         -6.5687e-03,  3.3622e-02, -4.1784e-03, -3.5189e-02,  1.5128e-02,
          1.8027e-02, -1.7781e-02,  3.8927e-04, -2.3789e-02, -1.9742e-02,
          3.5262e-02, -3.4977e-02, -2.3164e-02, -3.4642e-02,  2.9731e-02,
         -3.5738e-03,  8.0636e-04, -1.2149e-03, -4.3577e-03,  3.4910e-02,
          1.6013e-02, -3.9014e-03,  8.7278e-03,  1.1996e-02, -7.4370e-03,
         -1.5086e-02,  3.0152e-02, -6.9149e-04,  3.5315e-02,  1.5711e-02,

In [9]:
class BertTagger_ex(NERModel):
    
    def __init__(self, word_encoder):
        NERModel.__init__(self, word_encoder)
        self.lc_1 = nn.Linear(in_features=word_encoder.output_dim, out_features=20)
        self.drop = nn.Dropout()

    def train_forward(self, x):
        return self.forward(x)

    def forward(self, x):
        embedding = self.encode(x)
        logits = self.lc_1(embedding)  # [num_of_tokens, class_num]
        _, pred = torch.max(logits, 1)
        return logits, pred

In [10]:
model3 = BertTagger_ex(word_encoder)
state_dict = torch.load('lc_1')['state_dict']
own_state = model3.state_dict()
for name, param in state_dict.items():
    if name not in own_state:
        print(name)
        continue
    own_state[name].copy_(param)
print([(n, p) for n, p in model3.named_parameters() if n.startswith('lc')])

[('lc_1.weight', Parameter containing:
tensor([[ 2.8732e-02, -3.5536e-02, -1.8149e-02,  5.2805e-03,  2.0194e-04,
          1.6771e-02, -8.1999e-03, -3.2431e-02,  8.2918e-03,  3.5326e-02,
         -2.0378e-02,  3.4141e-03, -2.3877e-03, -3.0609e-02, -2.5336e-02,
          6.6329e-03, -2.6148e-02, -8.4295e-03,  2.6116e-02,  7.3001e-03,
          1.8179e-03,  8.5523e-03, -8.3591e-03, -2.2094e-02,  1.4133e-02,
          9.4152e-03, -9.1371e-03,  2.6205e-02,  9.9538e-03,  1.9322e-02,
         -2.0262e-02, -2.2829e-02,  1.0747e-02, -4.3306e-04,  6.9762e-03,
         -6.5687e-03,  3.3622e-02, -4.1784e-03, -3.5189e-02,  1.5128e-02,
          1.8027e-02, -1.7781e-02,  3.8927e-04, -2.3789e-02, -1.9742e-02,
          3.5262e-02, -3.4977e-02, -2.3164e-02, -3.4642e-02,  2.9731e-02,
         -3.5738e-03,  8.0636e-04, -1.2149e-03, -4.3577e-03,  3.4910e-02,
          1.6013e-02, -3.9014e-03,  8.7278e-03,  1.1996e-02, -7.4370e-03,
         -1.5086e-02,  3.0152e-02, -6.9149e-04,  3.5315e-02,  1.5711e-02,